# First model

In [1]:
import tensorflow as tf

assert tf.config.list_physical_devices('GPU')
assert tf.test.is_built_with_cuda()

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

trainDatagen = ImageDataGenerator(
    rescale=1./255
    #rotation_range= 40,
    #brightness_range=[0.2,1.0],
    #horizontal_flip=True
)
testDatagen = ImageDataGenerator(
    rescale=1./255)

In [3]:
category = "Gender"

In [5]:
trainData = trainDatagen.flow_from_directory(
    directory=category+"/train/", 
    class_mode="binary",
    target_size=(200,200),
    batch_size=32)
    
testData = testDatagen.flow_from_directory(
    directory=category+"/test/",
    class_mode="binary",
    target_size=(200,200),
    batch_size= 32)

Found 15584 images belonging to 2 classes.
Found 5453 images belonging to 2 classes.


In [6]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy, binary_crossentropy
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Activation
from tensorflow.keras import Sequential

In [7]:
model = Sequential([
    Conv2D(
        kernel_size = 4,
        filters=64,
        activation = "relu",
        input_shape = (200, 200, 3)),
    MaxPool2D(pool_size=2),
    Conv2D(10, 3, activation = "relu"),
    MaxPool2D(pool_size=2),
    Conv2D(10, 3, activation = "relu"),
    MaxPool2D(pool_size=2),
    Conv2D(10, 3, activation = "relu"),
    Flatten(),
    Dense(1, activation = "sigmoid")
])

In [8]:
model.compile(
    loss = binary_crossentropy,
    optimizer = Adam(),
    metrics = ["accuracy"]
)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 197, 197, 64)      3136      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 98, 98, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 10)        5770      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 10)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 10)        910       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 10)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 10)        9

In [10]:
history = model.fit(
    trainData,
    steps_per_epoch = len(trainData),
    validation_data = testData,
    validation_steps = len(testData),
    epochs=1000,
    callbacks = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience = 10)
)

Epoch 1/1000
487/487 [==============================] - 240s 447ms/step - loss: 0.4371 - accuracy: 0.7904 - val_loss: 0.3282 - val_accuracy: 0.8582
Epoch 2/1000
487/487 [==============================] - 131s 269ms/step - loss: 0.3038 - accuracy: 0.8780 - val_loss: 0.2890 - val_accuracy: 0.8872
Epoch 3/1000
487/487 [==============================] - 148s 303ms/step - loss: 0.2763 - accuracy: 0.8878 - val_loss: 0.2780 - val_accuracy: 0.8878
Epoch 4/1000
487/487 [==============================] - 139s 284ms/step - loss: 0.2517 - accuracy: 0.8975 - val_loss: 0.2957 - val_accuracy: 0.8834
Epoch 5/1000
487/487 [==============================] - 139s 286ms/step - loss: 0.2326 - accuracy: 0.9094 - val_loss: 0.2438 - val_accuracy: 0.9087
Epoch 6/1000
487/487 [==============================] - 143s 293ms/step - loss: 0.2202 - accuracy: 0.9162 - val_loss: 0.2932 - val_accuracy: 0.8802
Epoch 7/1000
487/487 [==============================] - 143s 293ms/step - loss: 0.2079 - accuracy: 0.9215 - val_

In [11]:
model.save("myModel3")

INFO:tensorflow:Assets written to: myModel3\assets


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
def predict(model, path):
    img = load_img(path, target_size=(200, 200))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    print(model.predict(img_array))

predict(model, "ja.jpg")